In [2]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch

# E5 임베딩 생성 클래스
class E5Embedder:
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained("intfloat/e5-large")
        self.model = AutoModel.from_pretrained("intfloat/e5-large")

    def get_embedding(self, text):
        """텍스트를 E5 임베딩 벡터로 변환"""
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True)
        outputs = self.model(**inputs)
        return outputs.last_hidden_state.mean(dim=1).detach()

# 데이터프레임 로드
df = pd.read_csv('new_trot_clean.csv')

# E5 임베딩 생성 및 저장
embedder = E5Embedder()

if 'embedding' not in df.columns:  # 최초 실행 시만 임베딩 생성
    print("임베딩 생성 중...")
    df['embedding'] = df['cleaned_lyrics'].apply(lambda x: embedder.get_embedding(x))
    df.to_pickle("./trot_embeddings.pkl")  # 임베딩 포함 데이터 저장
else:
    df = pd.read_pickle("./trot_embeddings.pkl")  # 이미 저장된 데이터 로드


임베딩 생성 중...


In [12]:
import torch
from transformers import BertForSequenceClassification, AutoTokenizer
import re

class EmotionClassifier:
    def __init__(self, model_path="monologg/kobert", num_labels=7, device=None):
        """
        KoBERT 모델 초기화
        """
        self.device = device if device else ("cuda" if torch.cuda.is_available() else "cpu")
        self.model = BertForSequenceClassification.from_pretrained(model_path, num_labels=num_labels)
        self.tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
        self.model.to(self.device)

        # 감정 매핑
        self.label_to_emotion = {
            0: "중립",
            1: "놀람",
            2: "분노",
            3: "슬픔",
            4: "행복",
            5: "혐오",
            6: "공포"
        }

    def load_model(self, model_file):
        """
        학습된 감정 분류 모델 불러오기
        """
        state_dict = torch.load(model_file, map_location=self.device)
        self.model.load_state_dict(state_dict)

    def preprocess_text(self, text):
        """
        텍스트 전처리
        """
        return re.sub("[^0-9a-zA-Z가-힣\s+]", "", text)

    def predict_emotion(self, text):
        """
        감정 분류 및 예측
        """
        # 텍스트 전처리 및 토큰화
        cleaned_text = self.preprocess_text(text)
        encoded_input = self.tokenizer(cleaned_text, return_tensors="pt", truncation=True, padding="max_length", max_length=128)
        encoded_input = {key: val.to(self.device) for key, val in encoded_input.items()}

        # KoBERT 모델로 감정 예측
        self.model.eval()
        with torch.no_grad():
            outputs = self.model(**encoded_input)
            predicted_label = outputs.logits.argmax(dim=1).item()

        # 라벨을 감정 이름으로 변환
        predicted_emotion = self.label_to_emotion[predicted_label]
        
        return predicted_emotion


In [14]:
import pandas as pd

# 트로트 데이터 로드
song_data_path = "trot_embeddings.pkl"  # 크롤링된 데이터 CSV 파일 경로
df = pd.read_pickle(song_data_path)

# 데이터 확인
print("데이터프레임 구조:")
print(df.head())

# KoBERT 기반 감정 분류 모델 초기화 및 학습된 가중치 로드
emotion_model_path = "new_data_test.pth"  # 학습된 모델 경로
emotion_classifier = EmotionClassifier()
emotion_classifier.load_model(emotion_model_path)

# 'emotion' 컬럼 추가
def classify_emotion(lyrics):
    try:
        return emotion_classifier.predict_emotion(lyrics)
    except Exception as e:
        print(f"오류 발생: {e}")
        return None

df['emotion'] = df['cleaned_lyrics'].apply(classify_emotion)

# 결과 확인
print("\n감정이 추가된 데이터프레임:")
print(df.head())

# 결과 CSV로 저장
df.to_pickle("./trot_embeddings_emotion.pkl")
print("\nPKL 파일로 저장 완료!")


데이터프레임 구조:
              title          artist  \
0             진정인가요             정서주   
1    멘토링 (Feat. 곽범)             이지요   
2            사랑의삼매경  누나둘 (Nunadool)   
3  유리꽃 (Cover Ver.)             김준영   
4           제목은 당신꽃              동후   

                                      cleaned_lyrics  \
0  미련없다 그 말이 진정인가요 냉정했던 그 마음이 진정인가요 바닷가를 거닐며 수놓았던...   
1  그댄 나의 멘토야 나는 당신의 에너지 즐거운 하루되세요 행복한 기분 너무 좋아요 그...   
2  삼매경에 빠져요 사랑에빠져요 삼매경에 빠져요 사랑의삼매경에 빠져요 흠뻑 빠져버린 내...   
3  유리꽃처럼 입술만 훔치고 가버린 그날 그 카페에 지울 수 없는 너의 향기가 연기처럼...   
4  가로등에 꽃이 피네요 그대와 함께 거닐 때 수줍어하던 그 사랑에 이름 모를 꽃이 피...   

                                           embedding  
0  [[tensor(-0.2948), tensor(-0.7847), tensor(0.6...  
1  [[tensor(-0.6234), tensor(-0.9389), tensor(0.8...  
2  [[tensor(-0.4987), tensor(-1.0133), tensor(0.5...  
3  [[tensor(-0.5342), tensor(-0.9054), tensor(0.6...  
4  [[tensor(-0.3140), tensor(-1.0779), tensor(0.8...  


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



감정이 추가된 데이터프레임:
              title          artist  \
0             진정인가요             정서주   
1    멘토링 (Feat. 곽범)             이지요   
2            사랑의삼매경  누나둘 (Nunadool)   
3  유리꽃 (Cover Ver.)             김준영   
4           제목은 당신꽃              동후   

                                      cleaned_lyrics  \
0  미련없다 그 말이 진정인가요 냉정했던 그 마음이 진정인가요 바닷가를 거닐며 수놓았던...   
1  그댄 나의 멘토야 나는 당신의 에너지 즐거운 하루되세요 행복한 기분 너무 좋아요 그...   
2  삼매경에 빠져요 사랑에빠져요 삼매경에 빠져요 사랑의삼매경에 빠져요 흠뻑 빠져버린 내...   
3  유리꽃처럼 입술만 훔치고 가버린 그날 그 카페에 지울 수 없는 너의 향기가 연기처럼...   
4  가로등에 꽃이 피네요 그대와 함께 거닐 때 수줍어하던 그 사랑에 이름 모를 꽃이 피...   

                                           embedding emotion  
0  [[tensor(-0.2948), tensor(-0.7847), tensor(0.6...      슬픔  
1  [[tensor(-0.6234), tensor(-0.9389), tensor(0.8...      행복  
2  [[tensor(-0.4987), tensor(-1.0133), tensor(0.5...      슬픔  
3  [[tensor(-0.5342), tensor(-0.9054), tensor(0.6...      슬픔  
4  [[tensor(-0.3140), tensor(-1.0779), tensor(0.8...      행복  

PKL 파일로 저장 완료!


In [15]:
df

,title,artist,cleaned_lyrics,embedding,emotion
0,진정인가요,정서주,미련없다 그 말이 진정인가요 냉정했던 그 마음이 진정인가요 바닷가를 거닐며 수놓았던...,"[[tensor(-0.2948), tensor(-0.7847), tensor(0.6...",슬픔
1,멘토링 (Feat. 곽범),이지요,그댄 나의 멘토야 나는 당신의 에너지 즐거운 하루되세요 행복한 기분 너무 좋아요 그...,"[[tensor(-0.6234), tensor(-0.9389), tensor(0.8...",행복
2,사랑의삼매경,누나둘 (Nunadool),삼매경에 빠져요 사랑에빠져요 삼매경에 빠져요 사랑의삼매경에 빠져요 흠뻑 빠져버린 내...,"[[tensor(-0.4987), tensor(-1.0133), tensor(0.5...",슬픔
3,유리꽃 (Cover Ver.),김준영,유리꽃처럼 입술만 훔치고 가버린 그날 그 카페에 지울 수 없는 너의 향기가 연기처럼...,"[[tensor(-0.5342), tensor(-0.9054), tensor(0.6...",슬픔
4,제목은 당신꽃,동후,가로등에 꽃이 피네요 그대와 함께 거닐 때 수줍어하던 그 사랑에 이름 모를 꽃이 피...,"[[tensor(-0.3140), tensor(-1.0779), tensor(0.8...",행복
...,...,...,...,...,...
340,내사랑 껌딱지,그린,어디 있나요 어디 갔나요 내 사랑 껌딱지 바라만 봐도 너무 행복해 내 사랑 껌딱지 ...,"[[tensor(-0.4370), tensor(-0.6369), tensor(0.2...",슬픔
341,봄날의 춘향이,김 소피아,따스한 봄바람 불어오네 춘향이는 꽃길을 걸어 그녀의 웃음 봄햇살 같아 모두의 시선 ...,"[[tensor(-0.5346), tensor(-0.9237), tensor(0.3...",행복
342,괜찮아,김민주,바람도 잠이 들고 달빛마저 숨어 버린 밤 무엇을 찾으려고 나 여기 어둠에 서있나 귓...,"[[tensor(-0.2623), tensor(-1.0824), tensor(0.5...",슬픔
343,자네,조재권,정신없이 달려왔는데 앞만보고 뛰어왔는데 하나둘씩 떠나고 나홀로 덩그러니 남아 있구나...,"[[tensor(-0.3968), tensor(-1.1507), tensor(0.4...",슬픔


In [23]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy

class SongRecommender:
    def __init__(self, df):
        """
        df: 노래 데이터가 담긴 Pandas 데이터프레임
            컬럼: ['title', 'artist', 'cleaned_lyrics', 'emotion', 'embedding']
        """
        self.df = df

    def recommend_song(self, diary_embedding, emotion):
        """
        감정이 동일한 노래 중에서 가장 유사한 노래 추천
        """
        # 감정이 동일한 노래 필터링
        filtered_df = self.df[self.df['emotion'] == emotion]

        # 코사인 유사도 계산
        similarities = []
        for _, row in filtered_df.iterrows():
            similarity = cosine_similarity(diary_embedding.numpy(), row['embedding'].numpy())
            similarities.append((row['title'], row['artist'], row['cleaned_lyrics'], similarity[0][0]))

        # 유사도 순으로 정렬하여 가장 유사한 노래 선택
        if similarities:
            best_match = sorted(similarities, key=lambda x: x[3], reverse=True)[0]
            return best_match  # (title, artist, lyrics, similarity)
        else:
            return None


In [24]:
if __name__ == "__main__":
    # 모델 경로 및 데이터 로드
    emotion_model_path = "monologg/kobert"

    # 감정 분류 모델 초기화
    emotion_classifier = EmotionClassifier(model_path=emotion_model_path)

    # 추천 시스템 초기화
    recommender = SongRecommender(df)

    # 감정 학습 모델 불러오기
    emotion_classifier.load_model("new_data_test.pth")

    # 사용자 입력 처리
    user_input = input("오늘의 일기를 입력하세요: ")

    # 감정 예측
    predicted_emotion = emotion_classifier.predict_emotion(user_input)

    # 사용자 입력 텍스트의 임베딩 생성
    diary_embedding = embedder.get_embedding(user_input)

    # 노래 추천 실행
    recommended_song = recommender.recommend_song(diary_embedding, predicted_emotion)

    # 결과 출력 (보기 좋게 포맷팅)
    print("\n--- 오늘의 추천 결과 ---")
    print(f"예측된 감정: {predicted_emotion}")

    if recommended_song:
        title, artist, lyrics, similarity = recommended_song
        print(f"🎵 제목: {title}")
        print(f"👤 가수: {artist}")
        print(f"📜 가사:\n{lyrics}")
        print(f"🔗 유사도 점수: {similarity:.4f}")
    else:
        print("추천할 노래가 없습니다.")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



--- 오늘의 추천 결과 ---
예측된 감정: 슬픔
🎵 제목: 어느 부부 이야기
👤 가수: 이영실
📜 가사:
해 저무는 곳에 사랑이 피면 나 행복할 수 있어요 해 저무는 곳에 꽃이 핀다면 나 슬퍼하지 않아요 어둠이 좋아 울 수 있어서 눈물도 보이지 않아 그 누가 있어 우리 사랑을 한 발자국 이끌어줄까 해 저무는 곳에 행복이 있다면 나 따라갈 수 있어요 어둠이 좋아 울 수 있어서 눈물도 보이지 않아 그 누가 있어 우리 사랑을 한 발자국 이끌어줄까 이끌어줄까 이끌어줄까 해 저무는 곳에 행복이 있다면 나 따라갈 수 있어요
🔗 유사도 점수: 0.8805
